In [35]:
import pandas as pd

# Load Data

In [36]:
ndc = pd.read_csv('data/ndc.csv')

In [37]:
label = pd.read_csv('data/label.csv')

# Data Cleaning

## Join the Files

In [38]:
df = ndc.merge(
    label,
    left_on='spl_id',
    right_on='id',
    how='left'
)

In [39]:
# delete the dataframes to save memory
del ndc 
del label

## Remove unfinished products because the FDA only reviews finished products. 

In [40]:
df = df.query('finished == True')

In [41]:
df.columns

Index(['product_id', 'product_ndc', 'spl_id', 'application_number',
       'dea_schedule', 'dosage_form', 'finished', 'marketing_category',
       'marketing_start_date', 'marketing_end_date', 'openfda.pharm_class_cs',
       'openfda.pharm_class_epc', 'openfda.pharm_class_pe',
       'openfda.pharm_class_moa', 'pharm_class', 'product_type', 'route', 'id',
       'set_id', 'version', 'effective_time', 'drug_interactions'],
      dtype='object')

## Select Columns 

Exclude IDs and other columns that won't be used as features due to bias detected during the exploration phase. 

In [42]:
cols_to_keep = [
    'application_number',
    'dea_schedule',
    'dosage_form',
    'marketing_start_date',
    'openfda.pharm_class_cs',
    'openfda.pharm_class_epc',
    'openfda.pharm_class_pe',
    'openfda.pharm_class_moa',
    'product_type',
    'route',
    'drug_interactions'
]

In [43]:
df = df[cols_to_keep]

## Derive Target

Derive target variable from application_number. All products with an application number have been approved. 

In [44]:
df['approved'] = df['application_number'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [45]:
df.drop(columns=['application_number'], inplace=True)

## Replace Nulls 

In [46]:
df.isna().sum() / len(df)

dea_schedule               0.955558
dosage_form                0.000000
marketing_start_date       0.000000
openfda.pharm_class_cs     0.852796
openfda.pharm_class_epc    0.730471
openfda.pharm_class_pe     0.879947
openfda.pharm_class_moa    0.871066
product_type               0.000000
route                      0.018502
drug_interactions          0.648037
approved                   0.000000
dtype: float64

All columns with nulls are categorical, so let's replace their nulls with a single category. 

In [47]:
for col in df.columns[df.isna().sum() > 0]:
    df[col].fillna('N/A', inplace=True)

In [49]:
df.isna().sum()

dea_schedule               0
dosage_form                0
marketing_start_date       0
openfda.pharm_class_cs     0
openfda.pharm_class_epc    0
openfda.pharm_class_pe     0
openfda.pharm_class_moa    0
product_type               0
route                      0
drug_interactions          0
approved                   0
dtype: int64

# Feature Transformation